# Proof-Carrying Symbolic Computation with Wolfram MCP

This notebook demonstrates capabilities that go far beyond simple command-line Mathematica usage. The key advantages:

1. **Persistent Sessions** - Variables and functions persist across calls
2. **Proof Provenance** - Track assumptions, extract conditions, verify transformations
3. **Typed Equalities** - Not just "are these equal?" but "*how* are they equal?"
4. **Obligation Engine** - Register tests that must pass before accepting a derivation
5. **Numeric Validation** - Automatic spot-checks catch symbolic manipulation errors
6. **Math CI** - Build regression test suites for mathematical derivations
7. **Semantic Diffs** - See what *mathematically* changed, not just textually

---

## Demo 1: The Danger of Naive Symbolic Computation

Consider this common trap. A user asks: "Simplify $\sqrt{x^2}$"

### Naive approach (command-line Mathematica):
```mathematica
In[1]:= Simplify[Sqrt[x^2]]
Out[1]= Sqrt[x^2]    (* Didn't simplify *)

In[2]:= PowerExpand[Sqrt[x^2]]
Out[2]= x            (* WRONG for x < 0! *)
```

### With proof-carrying computation:

**Correct approach with assumptions:**
```json
wolfram_eval_proven("Simplify[Sqrt[x^2]]", assumptions="x \u2208 Reals")

{
  "expression": "Abs[x]",
  "latex": "|x|",
  "assumptions": {
    "assumptions": "Element[x, Reals]",
    "dangerous_flags": []   // No dangerous operations!
  }
}
```

**Dangerous approach (flagged!):**
```json
wolfram_eval("PowerExpand[Sqrt[x^2]]")

Result: x
// WARNING: PowerExpand assumes all variables are positive reals!
```

The system explicitly tracks that `Abs[x]` is the correct answer, and would flag `PowerExpand` as using dangerous assumptions.

## Demo 2: Typed Equalities - More Than Just "Equal"

In physics and mathematics, there are many *types* of equality:

| Type | Meaning | Example |
|------|---------|----------|
| Exact | Always equal | $\sin^2 x + \cos^2 x = 1$ |
| On Domain | Equal where defined | $\frac{x^2-1}{x-1} = x+1$ for $x \neq 1$ |
| Up to Order | Series agreement | $e^x = 1 + x + \frac{x^2}{2} + O(x^3)$ |
| Modulo Gauge | Equal up to gauge transform | $A_\mu \sim A_\mu + \partial_\mu \chi$ |

### Real output:

```json
wolfram_typed_equality(
    lhs="(x^2 - 1)/(x - 1)",
    rhs="x + 1",
    equality_type="on_domain",
    domain="x != 1",
    variables="x"
)

{
  "equality": {
    "lhs": "(x^2 - 1)/(x - 1)",
    "rhs": "x + 1",
    "type": "EqualityOnDomain",
    "domain": "x != 1"
  },
  "numeric_verified": true
}
```

## Demo 3: Semantic Diff - What Actually Changed?

When refactoring a calculation, you want to know if the *mathematical content* changed or just the representation.

### Real output:

```json
wolfram_semantic_diff(
    expr1="(x^2 - 1)/(x - 1)",
    expr2="x + 1",
    canonicalization="Cancel"
)

{
  "canonical1": "1 + x",
  "canonical2": "1 + x",
  "equivalent": false,
  "difference_type": "terms_changed",
  "pole_changes": [
    "Poles changed: (x - 1,) -> ()"   // Removable singularity detected!
  ],
  "summary": "Expressions differ. Poles changed: (x - 1,) -> ()"
}
```

The system detects that while the canonical forms are identical, there's a **removable singularity** at x=1 in the first expression!

## Demo 4: Domain Inference - Know Where Your Expression Lives

Before using an expression, you should know where it's defined.

### Real output:

```json
wolfram_infer_domain(
    expression="Log[x] + 1/x + Sqrt[1-x]",
    variables="x"
)

{
  "name": "Domain_x",
  "base_set": "Reals",
  "constraints": [
    "0 < x <= 1"    // Must be positive for Log, <=1 for Sqrt
  ],
  "excluded_points": [
    "x <= 0 || x >= 1"   // Boundary issues
  ],
  "branch_cuts": [
    "Principal branch"   // From Sqrt
  ]
}
```

## Demo 5: The Obligation Engine - Physics-Style Derivation Checking

In theoretical physics, we often have **consistency conditions** that must hold: Ward identities, unitarity bounds, dimensional correctness. The obligation engine lets you register these as tests.

### Real output:

```json
// Register obligations
wolfram_register_obligation(
    name="trig_identity",
    description="Fundamental trig identity must hold",
    test_type="identity",
    test_expression="Sin[x]^2 + Cos[x]^2",
    expected="1"
)
// -> obligation_id: "cf9d9282"

wolfram_register_obligation(
    name="double_angle",
    description="Double angle formula",
    test_type="identity",
    test_expression="Sin[2x]",
    expected="2 Sin[x] Cos[x]"
)

wolfram_register_obligation(
    name="exp_positive",
    description="Exponential is always positive",
    test_type="positive",
    test_expression="Exp[x]"
)

// Check all obligations
wolfram_check_obligations()

{
  "total": 3,
  "passed": 3,
  "failed": 0,
  "results": [
    {"name": "trig_identity", "status": "passed", "result": "True"},
    {"name": "double_angle", "status": "passed", "result": "True"},
    {"name": "exp_positive", "status": "passed", "result": "True"}
  ]
}
```

## Demo 6: Math CI - Regression Tests for Derivations

Build test suites that verify mathematical identities. This is **continuous integration for mathematics**.

### Real output:

```json
wolfram_create_test_suite(
    name="Trig Identities",
    description="Verify fundamental trigonometric identities"
)

wolfram_add_test(suite_name="Trig Identities", test_name="pythagorean",
    category="identity", expression="Sin[x]^2 + Cos[x]^2", expected="1")
    
wolfram_add_test(suite_name="Trig Identities", test_name="double_angle_sin",
    category="identity", expression="Sin[2x]", expected="2 Sin[x] Cos[x]")
    
wolfram_add_test(suite_name="Trig Identities", test_name="double_angle_cos",
    category="identity", expression="Cos[2x]", expected="Cos[x]^2 - Sin[x]^2")
    
wolfram_add_test(suite_name="Trig Identities", test_name="tan_definition",
    category="identity", expression="Tan[x]", expected="Sin[x]/Cos[x]")

// Run all tests
wolfram_run_test_suite("Trig Identities")

{
  "suite": "Trig Identities",
  "total": 4,
  "passed": 4,
  "failed": 0,
  "results": [
    {"name": "pythagorean", "status": "passed"},
    {"name": "double_angle_sin", "status": "passed"},
    {"name": "double_angle_cos", "status": "passed"},
    {"name": "tan_definition", "status": "passed"}
  ]
}
```

## Demo 7: Persistent Session State

Unlike command-line Mathematica where each query is independent, this system maintains state.

### Real output:

```json
// Define a function
wolfram_define(
    name="myIntegral",
    definition="[a_, b_] := Integrate[x^2, {x, a, b}]"
)
// -> "Defined 'myIntegral' successfully."

// Use it later (no need to redefine!)
wolfram_eval("myIntegral[0, 3]")
// -> Result: 9, LaTeX: $9$

wolfram_eval("myIntegral[1, 5]")
// -> Result: 124/3, LaTeX: $\frac{124}{3}$

// Check session state
wolfram_session_info()

{
  "session_id": "default",
  "created": "2026-01-18T02:42:52",
  "defined_symbols": ["myIntegral", ...],
  "history_length": 5,
  "obligations_pending": 0,
  "test_suites": ["Trig Identities"],
  "expression_cache_size": 0
}
```

## Demo 8: Expression Hashing for Caching

Equivalent expressions get the same hash - useful for caching expensive computations.

### Real output:

```json
wolfram_expression_hash("1 + x + x^2")
// -> hash: "d2faf558c84344e6"

wolfram_expression_hash("x^2 + x + 1")
// -> hash: "d2faf558c84344e6"  // SAME! They're equivalent
// -> previously_seen: true

wolfram_expression_hash("x^2 + x + 2")
// -> hash: "...different..."  // Different expression
```

Also works with canonicalization:

```json
wolfram_canonicalize(
    expression="(x^3 - 1)/(x - 1)",
    method="rational"
)

{
  "original": "(x^3 - 1)/(x - 1)",
  "canonical": "1 + x + x^2",
  "hash": "698dcbb4e9eb1af9",
  "latex": "x^2+x+1"
}
```

## Demo 9: A Complete Physics Example

The integral $\int_0^\infty \frac{x^{s-1}}{e^x - 1} dx$ is related to the Riemann zeta function.

### Real output:

```json
wolfram_eval_proven(
    code="Integrate[x^(s-1)/(Exp[x]-1), {x, 0, Infinity}]",
    assumptions="Re[s] > 1",
    generate_conditions=true
)

{
  "expression": "Gamma[s]*PolyLog[s, 1]",
  "latex": "\\text{Li}_s(1) \\Gamma (s)",
  "assumptions": {
    "assumptions": "Re[s] > 1",
    "dangerous_flags": []
  },
  "timing": 1.636596,
  "hash": "c853dace3c236546"
}
```

Note: $\text{Li}_s(1) = \zeta(s)$, so the result is $\Gamma(s) \zeta(s)$.

**Verification at special values:**
```
wolfram_eval("{Gamma[2] * Zeta[2], Pi^2/6}")
// -> (Pi^2/6, Pi^2/6)  // Match!

wolfram_eval("{Gamma[4] * Zeta[4], Pi^4/15}")
// -> (Pi^4/15, Pi^4/15)  // Match!
```

---

## Summary: Why This Beats Command-Line Mathematica

| Feature | Command-Line | Proof-Carrying MCP |
|---------|--------------|--------------------|
| Session persistence | Each call independent | Full state persistence |
| Assumption tracking | Manual, error-prone | Automatic provenance |
| Condition extraction | Often ignored | Explicit in output |
| Numeric validation | Manual | Automatic spot-checks |
| Typed equalities | Just True/False | *How* they're equal |
| Test obligations | None | Physics-style constraints |
| Regression testing | None | Full test suites |
| Semantic diffs | None | Mathematical change detection |
| Domain inference | Manual | Automatic |
| Expression caching | None | Hash-based deduplication |

**This system turns Mathematica from a calculator into a proof assistant.**